In [3]:
from sklearn import model_selection, metrics
from sklearn import (svm, 
                     neighbors, 
                     ensemble, 
                    linear_model, 
                    naive_bayes, 
                    tree, 
                    discriminant_analysis)
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [2]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [3]:
import time

In [4]:
# train_data = pd.read_csv('./data/preprocessed_train_data.csv')
# test_data = pd.read_csv('./data/preprocessed_test_data.csv')
df = pd.read_csv('./data/preprocessed_train_test_data.csv', index_col=False)

In [17]:
train_data = df.loc[~df['sales'].isna()]
print("train_data", train_data.shape)
test_data = df.loc[df['sales'].isna()]
print("test_data", test_data.shape)

train_data (913000, 148)
test_data (45000, 148)


In [18]:
train_data.head()

,Unnamed: 0,date,store,item,sales,id,month,weekday,year,week_of_year,...,store_item_day_weekday_sales_sales_mean,store_item_day_dayofyear_sales_sum,store_item_day_dayofyear_sales_median,store_item_day_dayofyear_sales_mean,store_item_day_weekofyear_sales_sum,store_item_day_weekofyear_sales_median,store_item_day_weekofyear_sales_mean,store_item_year_month_day_weekofyear_dayofyear_weekday_sales_sum,store_item_year_month_day_weekofyear_dayofyear_weekday_sales_median,store_item_year_month_day_weekofyear_dayofyear_weekday_sales_mean
0,0,2013-01-01,1,1,13.0,NaN,1,1,2013,1,...,17.333333,73.0,13.0,14.6,33.0,11.0,11.00,13.0,13.0,13.0
1,1,2013-01-02,1,1,11.0,NaN,1,2,2013,1,...,16.666667,72.0,14.0,14.4,59.0,14.5,14.75,11.0,11.0,11.0
2,2,2013-01-03,1,1,14.0,NaN,1,3,2013,1,...,18.555556,63.0,12.0,12.6,51.0,12.5,12.75,14.0,14.0,14.0
3,3,2013-01-04,1,1,13.0,NaN,1,4,2013,1,...,20.222222,75.0,14.0,15.0,75.0,14.0,15.00,13.0,13.0,13.0
4,4,2013-01-05,1,1,10.0,NaN,1,5,2013,1,...,21.888889,67.0,14.0,13.4,53.0,13.0,13.25,10.0,10.0,10.0


# Baseline Models

In [8]:
print("Baseline Model:", 100 / len(train_data['sales'].unique().tolist()), '%')

Baseline Model: 0.4694835680751174 %


In [9]:
import math 
def SMAPE1(y_true, y_pred):
    sum_val = 0 
    for t, p in zip(y_true, y_pred):
        numerator = math.fabs(p - t)
        denominator = (math.fabs(t) + math.fabs(p)) / 2
        if denominator == 0:
            denominator = 0.0001
        val = numerator / denominator
        sum_val += val 
    
    return (sum_val / len(y_true)) * 100 

def SMAPE2(y_true, y_pred):
    sum_val = 0 
    for t, p in zip(y_true, y_pred):
        numerator = math.fabs(p - t)
        denominator = (math.fabs(t) + math.fabs(p))
        if denominator == 0:
            denominator = 0.0001
        val = numerator / denominator
        sum_val += val 
    
    return (sum_val / len(y_true)) * 100 

def SMAPE3(y_true, y_pred):
    sum_1 = 0
    sum_2 = 0
    for t, p in zip(y_true, y_pred):
        sum_1 += math.fabs(p - t)
        sum_2 += (t + p) 
    if sum_2 == 0:
        sum_2 = 0.000001
    return (sum_1 / sum_2)

# Train Test Split

In [10]:
sample_data = train_data.sample(10000)

In [19]:
train_data.columns

Index(['Unnamed: 0', 'date', 'store', 'item', 'sales', 'id', 'month',
       'weekday', 'year', 'week_of_year',
       ...
       'store_item_day_weekday_sales_sales_mean',
       'store_item_day_dayofyear_sales_sum',
       'store_item_day_dayofyear_sales_median',
       'store_item_day_dayofyear_sales_mean',
       'store_item_day_weekofyear_sales_sum',
       'store_item_day_weekofyear_sales_median',
       'store_item_day_weekofyear_sales_mean',
       'store_item_year_month_day_weekofyear_dayofyear_weekday_sales_sum',
       'store_item_year_month_day_weekofyear_dayofyear_weekday_sales_median',
       'store_item_year_month_day_weekofyear_dayofyear_weekday_sales_mean'],
      dtype='object', length=148)

In [20]:
target = ['sales']
features = np.delete(train_data.columns, [0,1,4, 5])

In [21]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(sample_data[features], sample_data[target], random_state=0)
train_y = train_y['sales']
test_y = test_y['sales']

In [22]:
# List of MLA Algorithms
MLA = [
    # ensemble
#     ensemble.AdaBoostClassifier(),
#     ensemble.AdaBoostRegressor(),
#     ensemble.RandomForestClassifier(), 
    ensemble.RandomForestRegressor(),
#     ensemble.BaggingClassifier(),
    ensemble.GradientBoostingRegressor(),
    ensemble.ExtraTreesRegressor(),
    
    #Nearest Neighbor
#     neighbors.KNeighborsClassifier(),
    
    # svm
    svm.LinearSVR(), 
    svm.SVR(), 
    svm.NuSVR(),
    
    
    # tree
#     tree.DecisionTreeClassifier(),
    tree.DecisionTreeRegressor(),    
]

In [23]:
def ml_training(MLA):
    pd_dataframe = pd.DataFrame(columns=['Name', 'Train_Score', 'Test_Score', 'SAMPE_Train', 'SAMPE_Test', 'Time', 'Parameters'])
    row_number = 0
    for alg in MLA:
        alg_name = alg.__class__.__name__
        print("starting:", alg_name)

        start_time = time.time()
        alg.fit(train_X, train_y)
        end_time = time.time()
        time_taken = end_time - start_time

        train_score = alg.score(train_X, train_y)
        test_score = alg.score(test_X, test_y)
        sampe_train = SMAPE1(alg.predict(train_X), train_y)
        sampe_test = SMAPE1(alg.predict(test_X), test_y)

        # add to pandas dataframe
        pd_dataframe.loc[row_number] = [alg_name, train_score, test_score, sampe_train, sampe_test, time_taken, alg.get_params()]
        row_number+=1
        
    pd_dataframe.sort_values(by=['SAMPE_Test'], ascending=False, inplace=True)
    print('done')
    return pd_dataframe

In [24]:
MLA_Compare = ml_training(MLA)

starting: RandomForestRegressor
starting: GradientBoostingRegressor
starting: ExtraTreesRegressor
starting: LinearSVR
starting: SVR
starting: NuSVR
starting: DecisionTreeRegressor
done


In [ ]:
MLA_Compare

,Name,Train_Score,Test_Score,SAMPE_Train,SAMPE_Test,Time,Parameters
5,NuSVR,0.034203,-0.012022,45.021207,45.435170,10.948712,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd..."
4,SVR,0.020049,-0.040736,43.873103,45.403585,14.631797,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd..."
3,LinearSVR,0.854623,0.850293,18.628471,18.342273,3.261643,"{'C': 1.0, 'dual': True, 'epsilon': 0.0, 'fit_..."
1,GradientBoostingRegressor,0.999975,0.999956,0.148332,0.155181,6.124994,"{'alpha': 0.9, 'criterion': 'friedman_mse', 'i..."
0,RandomForestRegressor,0.999981,0.999995,0.010588,0.012108,3.627799,"{'bootstrap': True, 'criterion': 'mse', 'max_d..."
2,ExtraTreesRegressor,1.000000,0.999982,0.000000,0.010774,0.848513,"{'bootstrap': False, 'criterion': 'mse', 'max_..."
6,DecisionTreeRegressor,1.000000,0.999987,0.000000,0.003430,0.661627,"{'criterion': 'mse', 'max_depth': None, 'max_f..."


In [26]:
best_clf = tree.DecisionTreeRegressor()
best_clf.fit(train_X, train_y)
print(best_clf.score(train_data[features], train_data['sales']))
print(SMAPE1(best_clf.predict(train_data[features]), train_data['sales']))

0.9759100293146691
3.805219587529634


In [29]:
print(best_clf.score(test_X, test_y))
print(SMAPE1(best_clf.predict(test_X), test_y))
print(SMAPE2(best_clf.predict(test_X), test_y))
print(SMAPE3(best_clf.predict(test_X), test_y))

0.9035019489120373
15.220878350118408
7.610439175059204
0.06389892800151672


# Submission

In [63]:
predictions = best_clf.predict(test_data[features])
pandas_id = [i for i in range(len(predictions))]

In [64]:
print(predictions)
print(f'{len(pandas_id)} {len(predictions)}')

[ 5 19 19 ... 30 16 41]
45000 45000


In [65]:
pd.read_csv('./data/sample_submission.csv.zip').head()

,id,sales
0,0,52
1,1,52
2,2,52
3,3,52
4,4,52


In [66]:
submission_dataframe = pd.DataFrame({'id':pandas_id, 'sales':predictions})

In [67]:
submission_dataframe.tail()

,id,sales
44995,44995,26
44996,44996,24
44997,44997,30
44998,44998,16
44999,44999,41


In [68]:
submission_dataframe.to_csv('./data/sumbission.csv', index=False)